In [86]:
import pandas as pd
import numpy as np
import requests as r
import re
from bs4 import BeautifulSoup as bs
import sqlite3

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [87]:
DB = "..\\Data\\hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

In [91]:
q = 'SELECT unique_id, player FROM player_list'
players = run_query(q)
players

,unique_id,player
0,aebisda01,David Aebischer
1,allenja01,Jake Allen
2,anderfr01,Frederik Andersen
3,andercr01,Craig Anderson
4,aubinje01,Jean-Sebastien Aubin
...,...,...
310,wilkide01,Derek Wilkinson
311,wreggke01,Ken Wregget
312,yorkal01,Allen York
313,youngwe01,Wendell Young


In [110]:
def salary(prenom,surnom):
    baseurl = 'http://www.hockeyzoneplus.com'
    searchurl = baseurl + '/search-results/filter?q={0}+{1}'.format(prenom,surnom)
    
    page = r.get(searchurl)
    soup = bs(page.text,'html.parser')
    link = soup.find_all('a',href=True,itemprop="url")[0]['href']

    page2 = r.get(baseurl+link)
    soup2 = bs(page2.text,'html.parser')
    table = soup2.find_all('div',class_="table-container")

    for i,eachrow in enumerate(table[0].find_all('tr')):
        columnz = []
        if i == 0:
            for col in eachrow.find_all('td'):
                columnz.append(col.text)
            tableDF = pd.DataFrame(columns=columnz)
        else:
            for col in eachrow.find_all('td'):
                columnz.append(col.text)
            tableDF = tableDF.append(dict(zip(tableDF.columns,columnz)),ignore_index=True)
    tableDF['Salary($US)'] = tableDF['Salary($US)'].str.strip('$').str.replace(',','')        
    tableDF = tableDF.dropna(subset=['NHLRank'])

    return tableDF.reset_index(drop=True)[:-1].copy()

In [123]:
for i,player in players.iterrows():
    name = player['player'].split(' ')
    try:
        player_wages = salary(name[0],name[1]).iloc[:,:2]
    except:
        print('Error with %s'%player['player'])
    player_wages.columns = ['Season','Salary']
    player_wages.iloc[:,:2].to_csv('salaries\\'+player['unique_id']+'.csv')

Error with Dan Cloutier
Error with Scott Fankhouser
Error with Jeff Glass
Error with Matt Hackett
Error with Adin Hill
Error with Dieter Kochan
Error with Reggie Lemelin
Error with Marek Mazanec
Error with Sam Montembeault
Error with Adam Munro
Error with Neill
Error with Maxime Ouellet
Error with Calvin Petersen
Error with Daren Puppa
Error with Philippe Sauve
Error with Peter Sidorkiewicz
Error with Cam Talbot
Error with Steve Valiquette
Error with Scott Wedgewood


In [131]:
q = 'SELECT unique_id, player FROM player_list WHERE player = "Calvin Petersen"'
players = run_query(q)
players

,unique_id,player
0,valiqst01,Steve Valiquette


In [132]:
player_wages = salary('Stephen','Valiquette').iloc[:,:2]
player_wages.columns = ['Season','Salary']
player_wages.iloc[:,:2].to_csv('salaries\\'+player['unique_id']+'.csv')